In [1]:
import os
import decoupler
import pandas as pd 
pd.options.mode.chained_assignment = None  # default='warn'
import scanpy as sc

In [8]:
import sys
sys.path.append('/lustre/groups/ml01/workspace/samantha.bening/Bachelor/')
from importlib import reload
import genereporter.grn_pipeline as ggModule
reload(ggModule)

gg = ggModule.GRNPipeline(wdir="/lustre/groups/ml01/workspace/samantha.bening/Bachelor/", 
                 adata="data2/veo_ibd_balanced.h5ad",
                 f_adj='src/SCENICfiles/TFtg_adj.csv',
                 f_reg='src/SCENICfiles/reg_full10k.csv')
gg

In [9]:
# read in expression matrix 
# set a working directory
wdir = "/lustre/groups/ml01/workspace/samantha.bening/Bachelor/"
os.chdir( wdir )

adata = gg.adata
adata

AnnData object with n_obs × n_vars = 176789 × 28162
    obs: 'sampID', 'LMUSampID', 'PatientID', 'AnatomicLocation', 'PoolID', 'Protocol', 'CellsLoaded', 'FreezeProtocol', 'AnatomicLocation_Relation', 'AgeYM', 'Sex', 'Inflammation', 'Viability', 'SampleDate', 'sample_LMU_info_Timepoint', 'Ischemia_time', 'n_counts', 'log_counts', 'n_genes', 'mt_frac', 'ribo_frac', 'hb_frac', 'celltype_l3', 'celltype_l1', 'celltype_l2', 'PatientID_genotype'
    uns: 'PatientID_colors', 'PatientID_genotype_colors', 'celltype_l1_colors', 'celltype_l2_colors', 'celltype_l3_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'unintegrated_X_umap'
    varm: 'PCs'
    layers: 'raw'
    obsp: 'connectivities', 'distances'

In [10]:
geneset_df = gg.get_genesets()
geneset_df

,geneset,genesymbol
0,REACTOME_ABC_FAMILY_PROTEINS_MEDIATED_TRANSPORT,ABCA10
1,REACTOME_ABC_FAMILY_PROTEINS_MEDIATED_TRANSPORT,ABCA12
2,REACTOME_ABC_FAMILY_PROTEINS_MEDIATED_TRANSPORT,ABCA2
3,REACTOME_ABC_FAMILY_PROTEINS_MEDIATED_TRANSPORT,ABCA3
4,REACTOME_ABC_FAMILY_PROTEINS_MEDIATED_TRANSPORT,ABCA4
...,...,...
112865,ZNF85_REGULON,ANKS3
112866,ZNF85_REGULON,RDM1
112867,ZNF85_REGULON,CHAF1A
112868,ZNF85_REGULON,HEY1


In [11]:
# run AUCell on REACTOME pathway and regulon genesets

decoupler.run_aucell(
    adata,
    geneset_df,
    source="geneset",
    target="genesymbol",
    use_raw=False,
)

adata

AnnData object with n_obs × n_vars = 176789 × 28162
    obs: 'sampID', 'LMUSampID', 'PatientID', 'AnatomicLocation', 'PoolID', 'Protocol', 'CellsLoaded', 'FreezeProtocol', 'AnatomicLocation_Relation', 'AgeYM', 'Sex', 'Inflammation', 'Viability', 'SampleDate', 'sample_LMU_info_Timepoint', 'Ischemia_time', 'n_counts', 'log_counts', 'n_genes', 'mt_frac', 'ribo_frac', 'hb_frac', 'celltype_l3', 'celltype_l1', 'celltype_l2', 'PatientID_genotype'
    uns: 'PatientID_colors', 'PatientID_genotype_colors', 'celltype_l1_colors', 'celltype_l2_colors', 'celltype_l3_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'unintegrated_X_umap', 'aucell_estimate'
    varm: 'PCs'
    layers: 'raw'
    obsp: 'connectivities', 'distances'

In [12]:
adata_copy = adata

In [15]:
adata.obsm["aucell_estimate"]

source,AHDC1_REGULON,ARID5B_REGULON,ARX_REGULON,ASCL1_REGULON,ATF4_REGULON,ATF5_REGULON,ATF7_REGULON,BACH1_REGULON,BATF3_REGULON,BCL11B_REGULON,...,ZNF616_REGULON,ZNF624_REGULON,ZNF667_REGULON,ZNF721_REGULON,ZNF730_REGULON,ZNF737_REGULON,ZNF813_REGULON,ZNF85_REGULON,ZSCAN12_REGULON,ZSCAN31_REGULON
CID006480-1:ACCTGTCGTGGTCCCA,0.049298,0.015341,0.0,0.026424,0.095704,0.017058,0.000000,0.020131,0.000000,0.003745,...,0.007536,0.000000,0.030198,0.0,0.000000,0.000000,0.139603,0.046228,0.000000,0.036935
CID006480-1:GTAGTACTCGAGCCTG,0.069595,0.033994,0.0,0.023897,0.083901,0.032870,0.000000,0.033696,0.000000,0.013918,...,0.038395,0.027214,0.000000,0.0,0.009329,0.007407,0.124250,0.000000,0.000000,0.000000
CID006480-1:GTGTGATGTGAGTTTC,0.052722,0.034457,0.0,0.037624,0.082805,0.025888,0.000000,0.031919,0.000000,0.015840,...,0.052378,0.000000,0.000000,0.0,0.000000,0.012611,0.127707,0.000000,0.003649,0.000000
CID006480-1:TTGGATGTCATCGTAG,0.047487,0.021158,0.0,0.033074,0.097209,0.022111,0.000000,0.023213,0.051465,0.011022,...,0.026046,0.000000,0.008643,0.0,0.000000,0.008602,0.078292,0.020593,0.000000,0.000000
CID006480-1:ACACCAACAAATCGGG,0.060659,0.022274,0.0,0.026599,0.116864,0.042465,0.000000,0.061862,0.000000,0.066305,...,0.000860,0.000000,0.000000,0.0,0.000000,0.011523,0.129436,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CID008847-1:TCGCTCATCATTGCGA,0.042458,0.017709,0.0,0.024869,0.119979,0.084959,0.007846,0.062745,0.000000,0.050334,...,0.000000,0.042960,0.095475,0.0,0.050683,0.000000,0.162989,0.000000,0.000000,0.000000
CID008847-1:TTCAGGATCGTTCCTG,0.054247,0.020369,0.0,0.032160,0.111844,0.102048,0.000000,0.074484,0.000000,0.059595,...,0.013295,0.000000,0.040569,0.0,0.000000,0.041179,0.126690,0.002288,0.000000,0.043699
CID008847-1:CCGTTCAAGCTAGCCC,0.054363,0.056027,0.0,0.026560,0.152294,0.085268,0.007572,0.049120,0.058543,0.042375,...,0.021347,0.000000,0.000000,0.0,0.043434,0.046303,0.134215,0.048945,0.000000,0.022339
CID008847-1:GTCCTCACAAGCACAG,0.043492,0.020952,0.0,0.023352,0.103209,0.074312,0.000000,0.052143,0.000000,0.053465,...,0.008682,0.000000,0.000000,0.0,0.000000,0.030877,0.138993,0.007008,0.002136,0.034532


In [16]:
adata.write_h5ad("data2/veo_ibd_balanced_aucell.h5ad")